In [3]:
from collections import namedtuple
import pickle
import sys
sys.path.append("../..")

Transition = namedtuple("Transition", ("state", "action", "next_state", "reward"))

with open("../training_data/training_data.pt", "rb") as f:
    training_data = pickle.load(f)

In [5]:
import numpy as np

rewards = np.load("../training_data/training_data_rewards.npy")

In [9]:
from game_state import state_to_index
import tqdm

q_table_train_data = []

for i, data in tqdm.tqdm(enumerate(training_data)):
    state, action, next_state, reward = data
    if next_state is None:
        continue
    index = state_to_index(state)
    next_index = state_to_index(next_state)
    q_table_train_data.append([index, action, next_index, rewards[i]])

383781it [03:28, 1838.46it/s]


In [10]:
numpy_data = np.array(q_table_train_data)

In [11]:
np.save("q_table_train_data.npy", numpy_data)

In [26]:
from agent_code.q_table.game_state import FeatureState
from agent_code.q_table.actions import ACTIONS


q_table = np.random.rand(FeatureState.feature_size(), len(ACTIONS))

In [27]:
from tqdm import trange

EPOCHS = 10

k = FeatureState.feature_size() * len(ACTIONS)
t = 0

for epoch in trange(EPOCHS):
    q_table_copy = np.copy(q_table)

    for data in np.random.permutation(numpy_data):
        t += 1
        index, action, next_index, reward = data
        if action is None:
            continue
        action_index = ACTIONS.index(action)
        old_q_value = q_table_copy[index, action_index]
        learning_rate = k / (k + t)
        new_q_value = old_q_value + learning_rate * (reward + max(q_table_copy[next_index]) - old_q_value)
        q_table[index, action_index] = new_q_value

100%|██████████| 10/10 [00:12<00:00,  1.27s/it]


In [28]:
np.save("my-saved-q-table.npy", q_table)